# 邮件事件信息提取推理

这个 notebook 用于加载 LoRA 微调后的模型进行邮件事件信息提取，支持对比基础模型和微调模型的输出结果。

## 1. 导入依赖

In [1]:
import os
os.environ['HF_HOME'] = '/macroverse/public/database/huggingface/hub'
import torch
import time
import json
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from typing import Dict, Tuple


/macroverse/public/dingsz/infer_optim/qwen_finetune/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. 配置参数

在这里修改你的模型路径和推理参数

In [3]:
# 模型配置
BASE_MODEL = "Qwen/Qwen2.5-7B-Instruct"  # 基础模型名称
LORA_MODEL = "/macroverse/public/dingsz/infer_optim/qwen_finetune/outputs/lora_model/final_model"  # LoRA模型路径，例如: "./output/checkpoint-100"

# 推理参数
MAX_NEW_TOKENS = 512  # 最大生成token数
TEMPERATURE = 0.7     # 温度参数
TOP_P = 0.9          # top_p采样参数

# 测试数据配置
TEST_FILE = None      # 测试数据文件路径（JSONL格式），例如: "../data/test.jsonl"
MAX_SAMPLES = 5       # 测试样本最大数量（用于快速测试）

print(f"基础模型: {BASE_MODEL}")
print(f"LoRA模型: {LORA_MODEL if LORA_MODEL else '未指定（仅使用基础模型）'}")

基础模型: Qwen/Qwen2.5-7B-Instruct
LoRA模型: /macroverse/public/dingsz/infer_optim/qwen_finetune/outputs/lora_model/final_model


## 3. 定义辅助函数

In [4]:
def load_model(base_model_name, lora_model_path=None):
    """
    加载模型和tokenizer
    
    Args:
        base_model_name: 基础模型名称
        lora_model_path: LoRA模型路径（如果为None则只加载基础模型）
    """
    print(f"加载基础模型: {base_model_name}")
    tokenizer = AutoTokenizer.from_pretrained(
        base_model_name,
        trust_remote_code=True
    )
    
    model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True
    )
    
    # 如果提供了LoRA模型路径，则加载LoRA权重
    if lora_model_path:
        print(f"加载LoRA权重: {lora_model_path}")
        model = PeftModel.from_pretrained(model, lora_model_path)
        model = model.merge_and_unload()  # 合并LoRA权重到基础模型
    
    model.eval()
    return model, tokenizer

In [5]:
def extract_event_info(email_content, model, tokenizer, max_new_tokens=512, temperature=0.7, top_p=0.9):
    """
    从邮件中提取事件信息
    
    Args:
        email_content: 邮件内容
        model: 模型
        tokenizer: tokenizer
        max_new_tokens: 最大生成token数
        temperature: 温度参数
        top_p: top_p采样参数
    
    Returns:
        response: 提取的事件信息
        inference_time: 推理时间（秒）
    """
    start_time = time.time()
    
    # 构建对话消息
    messages = [
        {"role": "system", "content": "你是一个专业的邮件事件信息提取助手。"},
        {"role": "user", "content": f"请从以下邮件中提取事件信息，包括标题、时间、地点、参与者等关键信息，以JSON格式输出。\n\n邮件内容：\n{email_content}"}
    ]
    
    # 使用chat template格式化
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    # Tokenize
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    
    # 生成
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    # 解码输出
    response = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
    
    inference_time = time.time() - start_time
    
    return response, inference_time

In [6]:
def compare_outputs(base_output: str, finetuned_output: str, ground_truth: str = None) -> Dict:
    """
    对比基础模型和微调模型的输出
    
    Args:
        base_output: 基础模型的输出
        finetuned_output: 微调模型的输出
        ground_truth: 真实标注（可选）
    
    Returns:
        对比结果字典
    """
    result = {
        'base_valid_json': False,
        'finetuned_valid_json': False,
        'base_parsed': None,
        'finetuned_parsed': None,
        'ground_truth_parsed': None,
        'differences': []
    }
    
    # 解析基础模型输出
    try:
        result['base_parsed'] = json.loads(base_output)
        result['base_valid_json'] = True
    except json.JSONDecodeError:
        result['base_parsed'] = None
    
    # 解析微调模型输出
    try:
        result['finetuned_parsed'] = json.loads(finetuned_output)
        result['finetuned_valid_json'] = True
    except json.JSONDecodeError:
        result['finetuned_parsed'] = None
    
    # 解析真实标注
    if ground_truth:
        try:
            result['ground_truth_parsed'] = json.loads(ground_truth)
        except json.JSONDecodeError:
            result['ground_truth_parsed'] = None
    
    # 如果两个模型都成功解析，对比字段差异
    if result['base_parsed'] and result['finetuned_parsed']:
        base_dict = result['base_parsed']
        ft_dict = result['finetuned_parsed']
        
        all_keys = set(base_dict.keys()) | set(ft_dict.keys())
        for key in all_keys:
            base_val = base_dict.get(key, '(缺失)')
            ft_val = ft_dict.get(key, '(缺失)')
            
            if base_val != ft_val:
                diff_item = {
                    'field': key,
                    'base_value': base_val,
                    'finetuned_value': ft_val
                }
                
                # 如果有真实标注，添加对比
                if result['ground_truth_parsed'] and key in result['ground_truth_parsed']:
                    diff_item['ground_truth'] = result['ground_truth_parsed'][key]
                    diff_item['base_correct'] = base_val == result['ground_truth_parsed'][key]
                    diff_item['finetuned_correct'] = ft_val == result['ground_truth_parsed'][key]
                
                result['differences'].append(diff_item)
    
    return result

In [7]:
def print_comparison(comparison: Dict, base_time: float, ft_time: float):
    """
    打印对比结果
    
    Args:
        comparison: 对比结果字典
        base_time: 基础模型推理时间
        ft_time: 微调模型推理时间
    """
    print("\n" + "=" * 80)
    print("对比结果")
    print("=" * 80)
    
    # JSON格式正确性
    print(f"\nJSON格式:")
    print(f"  基础模型: {'✓ 正确' if comparison['base_valid_json'] else '✗ 错误'}")
    print(f"  微调模型: {'✓ 正确' if comparison['finetuned_valid_json'] else '✗ 错误'}")
    
    # 推理时间
    print(f"\n推理时间:")
    print(f"  基础模型: {base_time:.2f}秒")
    print(f"  微调模型: {ft_time:.2f}秒")
    print(f"  速度变化: {((ft_time - base_time) / base_time * 100):+.1f}%")
    
    # 字段差异
    if comparison['differences']:
        print(f"\n字段差异 ({len(comparison['differences'])}个):")
        print("-" * 80)
        for diff in comparison['differences']:
            print(f"\n  字段: {diff['field']}")
            print(f"    基础模型: {diff['base_value']}")
            print(f"    微调模型: {diff['finetuned_value']}")
            
            if 'ground_truth' in diff:
                print(f"    真实标注: {diff['ground_truth']}")
                print(f"    基础模型准确: {'✓' if diff['base_correct'] else '✗'}")
                print(f"    微调模型准确: {'✓' if diff['finetuned_correct'] else '✗'}")
    else:
        print("\n字段差异: 无差异（输出相同）")
    
    print("\n" + "=" * 80)

## 4. 加载模型

根据上面的配置加载模型（这个步骤会花费一些时间）

In [8]:
print("=" * 50)
print("开始加载模型...")
print("=" * 50)

model, tokenizer = load_model(BASE_MODEL, LORA_MODEL)

print("\n✓ 模型加载完成！")
print("=" * 50)

开始加载模型...
加载基础模型: Qwen/Qwen2.5-7B-Instruct


`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 4/4 [00:15<00:00,  3.82s/it]


加载LoRA权重: /macroverse/public/dingsz/infer_optim/qwen_finetune/outputs/lora_model/final_model

✓ 模型加载完成！


## 5. 单个样本测试

使用一个示例邮件进行测试

In [9]:
# 定义测试邮件
test_email = """主题：项目评审会议
发件人：项目经理 张三
收件人：开发团队

各位同事，

定于本周五（12月29日）下午3点在会议室B召开项目中期评审会议。请技术负责人和架构师务必参加，并准备项目进展汇报材料。

谢谢！
张三"""

print("测试邮件：")
print(test_email)
print("\n" + "=" * 50)
print("提取中...")

result, inference_time = extract_event_info(
    test_email, 
    model, 
    tokenizer, 
    MAX_NEW_TOKENS,
    TEMPERATURE,
    TOP_P
)

print(f"\n提取结果：\n{result}")
print(f"\n推理时间: {inference_time:.2f}秒")

# 验证JSON格式
try:
    parsed_result = json.loads(result)
    print("✓ JSON格式正确")
    print("\n格式化的JSON：")
    print(json.dumps(parsed_result, ensure_ascii=False, indent=2))
except json.JSONDecodeError as e:
    print(f"✗ JSON格式错误: {e}")

print("=" * 50)

测试邮件：
主题：项目评审会议
发件人：项目经理 张三
收件人：开发团队

各位同事，

定于本周五（12月29日）下午3点在会议室B召开项目中期评审会议。请技术负责人和架构师务必参加，并准备项目进展汇报材料。

谢谢！
张三

提取中...

提取结果：
{
  "event_type": "会议",
  "title": "项目中期评审会议",
  "time": "本周五（12月29日）下午3点",
  "location": "会议室B",
  "participants": ["技术负责人", "架构师"],
  "organizer": "项目经理 张三"
}

推理时间: 3.29秒
✓ JSON格式正确

格式化的JSON：
{
  "event_type": "会议",
  "title": "项目中期评审会议",
  "time": "本周五（12月29日）下午3点",
  "location": "会议室B",
  "participants": [
    "技术负责人",
    "架构师"
  ],
  "organizer": "项目经理 张三"
}


## 6. 自定义邮件测试

在这里输入你自己的邮件内容进行测试

In [ ]:
# 修改这里的邮件内容
custom_email = """主题：团建活动通知
发件人：HR部门
收件人：全体员工

大家好，

公司将于下周六（1月15日）上午10点组织团建活动，地点在市郊的绿野公园。请大家准时参加，穿着运动服装。

活动内容包括户外拓展训练和午餐聚会。

期待大家的参与！
HR部门"""

print("自定义邮件：")
print(custom_email)
print("\n" + "=" * 50)
print("提取中...")

result, inference_time = extract_event_info(
    custom_email, 
    model, 
    tokenizer, 
    MAX_NEW_TOKENS,
    TEMPERATURE,
    TOP_P
)

print(f"\n提取结果：\n{result}")
print(f"\n推理时间: {inference_time:.2f}秒")

# 验证JSON格式
try:
    parsed_result = json.loads(result)
    print("✓ JSON格式正确")
    print("\n格式化的JSON：")
    print(json.dumps(parsed_result, ensure_ascii=False, indent=2))
except json.JSONDecodeError as e:
    print(f"✗ JSON格式错误: {e}")

print("=" * 50)

## 7. 批量测试（可选）

如果你有测试数据文件，可以在这里进行批量测试

In [ ]:
# 设置测试数据文件路径
test_file_path = TEST_FILE  # 修改为你的测试文件路径，例如: "../data/test.jsonl"

if test_file_path and os.path.exists(test_file_path):
    print(f"从文件读取测试邮件: {test_file_path}")
    with open(test_file_path, 'r', encoding='utf-8') as f:
        test_data = [json.loads(line) for line in f]
    
    print(f"共 {len(test_data)} 条测试样本\n")
    
    total_inference_time = 0
    valid_json_count = 0
    
    # 限制测试样本数量
    samples_to_test = test_data[:MAX_SAMPLES] if MAX_SAMPLES else test_data
    
    for i, item in enumerate(samples_to_test):
        print(f"\n{'=' * 50}")
        print(f"测试样本 {i + 1}/{len(samples_to_test)}")
        print(f"{'=' * 50}")
        
        # 支持不同数据格式
        if 'messages' in item:
            email_content = item['messages'][1]['content'].split('邮件内容：\n')[-1]
            expected_output = item['messages'][2]['content']
        elif 'input' in item:
            email_content = item['input']
            expected_output = item.get('output')
        else:
            print("⚠️  数据格式不支持，跳过")
            continue
        
        print(f"\n输入邮件：\n{email_content[:200]}..." if len(email_content) > 200 else f"\n输入邮件：\n{email_content}")
        
        result, inference_time = extract_event_info(
            email_content, 
            model, 
            tokenizer, 
            MAX_NEW_TOKENS,
            TEMPERATURE,
            TOP_P
        )
        total_inference_time += inference_time
        
        print(f"\n模型输出：\n{result}")
        print(f"\n推理时间: {inference_time:.2f}秒")
        
        # 验证JSON格式
        try:
            json.loads(result)
            valid_json_count += 1
            print("✓ JSON格式正确")
        except json.JSONDecodeError:
            print("✗ JSON格式错误")
    
    # 统计信息
    avg_time = total_inference_time / len(samples_to_test)
    json_accuracy = valid_json_count / len(samples_to_test) * 100
    
    print(f"\n{'=' * 50}")
    print("推理统计:")
    print(f"{'=' * 50}")
    print(f"平均推理时间: {avg_time:.2f}秒/样本")
    print(f"JSON格式正确率: {json_accuracy:.1f}%")
    print(f"{'=' * 50}")
else:
    print(f"测试文件不存在或未指定: {test_file_path}")
    print("请在上面的配置中设置 TEST_FILE 变量为有效的测试数据文件路径")

## 8. 对比测试（可选）

如果你想对比基础模型和微调模型的效果，可以运行这个部分

In [10]:
# 对比模式需要重新加载两个模型
# 设置对比模式的配置
COMPARE_MODE = True  # 设置为 True 启用对比模式
LORA_MODEL_FOR_COMPARE = "/macroverse/public/dingsz/infer_optim/qwen_finetune/outputs/lora_model/final_model"  # 设置你的LoRA模型路径

if COMPARE_MODE and LORA_MODEL_FOR_COMPARE:
    print("\n对比模式：同时加载基础模型和微调模型")
    print(f"基础模型: {BASE_MODEL}")
    print(f"微调模型: {LORA_MODEL_FOR_COMPARE}")
    
    # 加载基础模型
    print("\n[1/2] 加载基础模型...")
    base_model, base_tokenizer = load_model(BASE_MODEL, lora_model_path=None)
    
    # 加载微调模型
    print("\n[2/2] 加载微调模型...")
    ft_model, ft_tokenizer = load_model(BASE_MODEL, LORA_MODEL_FOR_COMPARE)
    
    print("\n✓ 两个模型加载完成！")
    print("=" * 50)
    
    # 使用测试邮件进行对比
    email_content = test_email
    
    print(f"\n测试邮件:\n{email_content}")
    print("\n" + "=" * 80)
    
    # 基础模型推理
    print("[1/2] 基础模型推理中...")
    base_result, base_time = extract_event_info(
        email_content, 
        base_model, 
        base_tokenizer, 
        MAX_NEW_TOKENS,
        TEMPERATURE,
        TOP_P
    )
    
    # 微调模型推理
    print("[2/2] 微调模型推理中...")
    ft_result, ft_time = extract_event_info(
        email_content, 
        ft_model, 
        ft_tokenizer, 
        MAX_NEW_TOKENS,
        TEMPERATURE,
        TOP_P
    )
    
    # 显示原始输出
    print(f"\n基础模型输出:\n{base_result}")
    print(f"\n微调模型输出:\n{ft_result}")
    
    # 对比输出
    comparison = compare_outputs(base_result, ft_result)
    print_comparison(comparison, base_time, ft_time)
    
elif COMPARE_MODE:
    print("⚠️  对比模式需要设置 LORA_MODEL_FOR_COMPARE")
else:
    print("对比模式未启用。如需启用，请设置 COMPARE_MODE = True 和 LORA_MODEL_FOR_COMPARE")


对比模式：同时加载基础模型和微调模型
基础模型: Qwen/Qwen2.5-7B-Instruct
微调模型: /macroverse/public/dingsz/infer_optim/qwen_finetune/outputs/lora_model/final_model

[1/2] 加载基础模型...
加载基础模型: Qwen/Qwen2.5-7B-Instruct


Loading checkpoint shards: 100%|██████████| 4/4 [00:15<00:00,  3.83s/it]



[2/2] 加载微调模型...
加载基础模型: Qwen/Qwen2.5-7B-Instruct


Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.47s/it]


加载LoRA权重: /macroverse/public/dingsz/infer_optim/qwen_finetune/outputs/lora_model/final_model

✓ 两个模型加载完成！

测试邮件:
主题：项目评审会议
发件人：项目经理 张三
收件人：开发团队

各位同事，

定于本周五（12月29日）下午3点在会议室B召开项目中期评审会议。请技术负责人和架构师务必参加，并准备项目进展汇报材料。

谢谢！
张三

[1/2] 基础模型推理中...
[2/2] 微调模型推理中...

基础模型输出:
```json
{
  "标题": "项目评审会议",
  "时间": "12月29日 下午3点",
  "地点": "会议室B",
  "参与者": [
    {
      "角色": "技术负责人",
      "备注": "需准备项目进展汇报材料"
    },
    {
      "角色": "架构师",
      "备注": "需准备项目进展汇报材料"
    }
  ],
  "发起人": "项目经理 张三",
  "接收人": "开发团队"
}
```

微调模型输出:
{
  "event_type": "会议",
  "title": "项目中期评审会议",
  "time": "本周五（12月29日）下午3点",
  "location": "会议室B",
  "participants": ["开发团队的技术负责人", "架构师"],
  "organizer": "项目经理 张三"
}

对比结果

JSON格式:
  基础模型: ✗ 错误
  微调模型: ✓ 正确

推理时间:
  基础模型: 3.27秒
  微调模型: 2.10秒
  速度变化: -35.5%

字段差异: 无差异（输出相同）



## 9. 释放显存（可选）

如果需要释放GPU显存，可以运行这个单元格

In [11]:
import gc

# 删除模型对象
if 'model' in globals():
    del model
if 'tokenizer' in globals():
    del tokenizer
if 'base_model' in globals():
    del base_model
if 'base_tokenizer' in globals():
    del base_tokenizer
if 'ft_model' in globals():
    del ft_model
if 'ft_tokenizer' in globals():
    del ft_tokenizer

# 清理缓存
gc.collect()
torch.cuda.empty_cache()

print("✓ 显存已释放")

✓ 显存已释放
